# 🌀 Animation avec rebond corrigé contre les murs

In [ ]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv("trajectories.csv")

# Vérifie la colonne radius
if 'radius' in df.columns:
    df["size"] = df["radius"] * 100
else:
    df["size"] = df["id"].astype(int) % 5 * 4 + 10

df["id"] = df["id"].astype(str)

# Légère marge pour les rebonds sur les bords
xmin, xmax = -0.05, 1.05
ymin, ymax = -0.05, 1.05


In [ ]:
frames = []
for t in sorted(df["time"].unique()):
    step = df[df["time"] == t]
    shapes = []
    for _, row in step.iterrows():
        r = row["radius"] if "radius" in row else 0.02
        circle = dict(
            type="circle",
            xref="x", yref="y",
            x0=row["x"] - r,
            y0=row["y"] - r,
            x1=row["x"] + r,
            y1=row["y"] + r,
            line_color="rgba(0,0,150,0.3)",
            fillcolor="rgba(100,180,255,0.4)"
        )
        shapes.append(circle)

    trace = go.Scatter(
        x=step["x"], y=step["y"],
        mode="markers+text",
        marker=dict(size=step["size"], sizemode="diameter", sizeref=1,
                    color=step["id"].astype(int), colorscale="Viridis"),
        text=step["id"],
        hoverinfo="text"
    )
    frames.append(go.Frame(data=[trace], layout=go.Layout(shapes=shapes), name=str(t)))


In [ ]:
layout = go.Layout(
    title="Animation des particules avec rebond corrigé (mur)",
    xaxis=dict(range=[xmin, xmax], title="X"),
    yaxis=dict(range=[ymin, ymax], title="Y", scaleanchor="x", scaleratio=1),
    width=750, height=750,
    updatemenus=[dict(type="buttons", showactive=True,
                      buttons=[dict(label="Play", method="animate",
                                    args=[None, {"frame": {"duration": 1, "redraw": True},
                                                 "fromcurrent": True, "mode": "immediate"}]),
                               dict(label="Pause", method="animate",
                                    args=[[None], {"frame": {"duration": 0}, "mode": "immediate"}])
                              ])]
)

fig = go.Figure(data=frames[0].data, frames=frames, layout=layout)
fig.show()
